Install the required libraries in the cell below

In [57]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from pymongo import MongoClient, UpdateOne
import urllib
from tqdm import tqdm
from itertools import islice
import time
import numpy as np
from fake_useragent import UserAgent
import random
from recipe_scrapers import scrape_me
import os

In [58]:
#checking how a random user agent looks like
ua = UserAgent()
ua.random

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36'

Get the content from main recipes page (www.allrecipes.com/recipes)

In [ ]:
headers = {'user-agent':ua.random,}
url = "https://www.allrecipes.com/recipes"
r = requests.get(url)
print(r.content)

Use BeautifulSoup to structure the content from get request 

In [ ]:
soup = BeautifulSoup(r.content,'html5lib')
print(soup.prettify)

Store the class that contains all recipe category urls into table variable

In [22]:
table = soup.find('div', attrs = {'class':'col-container'}) 
print(table.prettify)

<bound method Tag.prettify of <div class="col-container">

                    <div class="all-categories-col">

                        <section>
                            <div )="" class="img-header" style="background-image: url(https://images.media-allrecipes.com/images/77855.jpg"></div>
                            <h3 class="heading__h3">Meal Type</h3>
                            <ul>
                                <li><a href="https://www.allrecipes.com/recipes/78/breakfast-and-brunch/">Breakfast and Brunch</a></li>
                                <li><a href="https://www.allrecipes.com/recipes/79/desserts/">Desserts</a></li>
                                <li><a href="https://www.allrecipes.com/recipes/17562/dinner/">Dinners</a></li>
                                <li><a href="https://www.allrecipes.com/recipes/17561/lunch/">Lunch</a></li>

                            </ul>
                        </section>


                        <section>
                            <di

Generate random UserAgent and define categories to scrape

In [59]:
ua =  UserAgent()

In [82]:
to_scrape = {'Vegetable Recipes','Chinese','Indian','Italian','Mexican','Southern','Thai',
             'All World Cuisine', 'Diabetic', 'Low Carb Recipes'}

Function to scrape urls of individual recipes from 50 pages for given parent url  

In [71]:
def get_recipe_urls(par_url, counter = 0):
    urls = set()
    for i in tqdm(range(51)):
        if i == 1:
            url = par_url
        else:
            url = par_url+"/?page="+str(i)
            #print(f'page url: {url}')
            
        headers = {'user-agent':ua.random,}
        try:
            r = requests.get(url, headers=headers,timeout=30)
        except Exception as e:
            print(str(e))
            print(f'proxy at {indx} could not connect')
            
        time.sleep(10)
        counter += 1
        if counter == 1000:
            time.sleep(3000)
            counter = 0
            
        soup = BeautifulSoup(r.content,'html5lib')
        table = soup.find('section', attrs = {'class':'recipe-section'}) 
        try:
            for row in table.findAll('div', attrs={'class':'fixed-recipe-card__info'}):
                urls.add(row.a['href'])
        except:
            print(f'No data for page {i}')            
            break
    #print(urls)
    return list(urls)
 

This function is to store urls in a dictionary 'recipes'. The keys in recipes are parent category of recipes. The values of each key correspond to the urls of individual recipes across 50 pages in allrecipes.com. To get urls get_recipe_urls function is called and parent category url is passed as parameter.

In [ ]:
recipes = {}
i = 0
for cat in table.findAll('div', attrs={'class':'all-categories-col'}):
    for item in cat.findAll('li'):
        url = item.a['href']
        key = item.a.text
        if key in to_scrape:
            try:
                if url.startswith('www.allrecipes',8):
                    recipes[key] = get_recipe_urls(url)
            except Exception as e:
                print(f'Invalid url - {url}')
                print(str(e))
        if i < 5:
            print(item.a['href'], item.a.text)
            i += 1
print(len(recipes))           

Checking the length of recipes

In [73]:
print(len(recipes))

14


7. It is recommended to run the cell below and save the urls in a json file. In case, some error occurs and we need to restart jupyter notebook, the url list can be loaded from the saved json.

In [74]:
with open('recipes.json', 'w') as fp:
    json.dump(recipes, fp)

Run the cell below only if we lose urls in recipes variable. If we save them in json, we can load from it instead of beginning again. Do not run this if recipes are not stored in recipes.json file

In [2]:
with open('recipes.json') as fp: 
    recipes = json.load(fp) 

To check the total number of urls scraped.

In [75]:
len_recipes = 0
for key in recipes:
    len_recipes += len(recipes[key])
print(f'Total number of recipes: {len_recipes}')

Total number of recipes: 11741


To remove if any duplicate urls are scraped under each category

In [76]:
recipes_unique = {key:list(set(recipes[key])) for key in recipes}


To check if the total number of urls scraped are same as unique urls

In [77]:
len_recipes = 0
for key in recipes_unique:
    len_recipes += len(recipes_unique[key])
print(f'Total number of unique recipes: {len_recipes}')

Total number of unique recipes: 11741


To check the categories scraped.

In [79]:
print(recipes_unique.keys())

dict_keys(['Breakfast and Brunch', 'Desserts', 'Dinners', 'Lunch', 'Beef', 'Beans and Legumes', 'Chicken Recipes', 'Chocolate', 'Fruit', 'Game Meats', 'Grains', 'Mushrooms', 'Tofu and Tempeh', 'Turkey'])


To check how the output from scrape_me fuction looks like.

In [59]:
for cat in recipes:
    for url in recipes[j]:
        scraper = scrape_me(url)
        print(f'Title: {scraper.title()}')
        print(f'Preparation Time: {scraper.total_time()}')
        print(f'Serves: {scraper.yields()}')
        print(f'Ingredients:{scraper.ingredients()}')
        print(f'Instructions: {scraper.instructions()}')
        print(f'Image url: {scraper.image()}')
        print(f'Category: {j}')

Title: Guacamole
Preparation Time: 10
Serves: None
Ingredients:['3 avocados - peeled, pitted, and mashed', '1 lime, juiced', '1 teaspoon salt', '½ cup diced onion', '3 tablespoons chopped fresh cilantro', '2 roma (plum) tomatoes, diced', '1 teaspoon minced garlic', '1 pinch ground cayenne pepper (optional)']
Instructions: In a medium bowl, mash together the avocados, lime juice, and salt. Mix in onion, cilantro, tomatoes, and garlic. Stir in cayenne pepper. Refrigerate 1 hour for best flavor, or serve immediately.
Image url: https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.com%2Fuserphotos%2F4521556.jpg
Category: 4th of July
Title: Red, White and Blue Deviled Eggs
Preparation Time: 60
Serves: 16 egg halves
Ingredients:['3 drops red gel food coloring', '⅔ cup water, or as needed', '2 drops blue gel food coloring', 'toothpicks', '8 hard-cooked eggs, peeled', '2 ½ tablespoons mayonnaise', '1 tablespoon hot prepared horseradish', '1 teaspoon rice vineg

Title: Garlic Corn on the Cob
Preparation Time: 40
Serves: 12 ears
Ingredients:['12 ears corn, husked and cleaned', '12 tablespoons butter, divided', '¼ cup garlic powder']
Instructions: Preheat grill for medium heat, or preheat your oven to 350 degrees F (175 degrees C).
Place each ear of corn on a separate square of aluminum foil. Place 1 tablespoon of butter on each one, and sprinkle with garlic powder. Wrap ears tightly with the foil.
Place ears of corn on the grill or in the oven for 20 to 30 minutes, turning over occasionally.
Image url: https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.com%2Fuserphotos%2F225775.jpg
Category: 4th of July
Title: Instant Pot&reg; Baked Beans
Preparation Time: 130
Serves: None
Ingredients:['8 cups water', '1 pound dry navy beans, rinsed and picked through', '1 tablespoon olive oil', '6 ounces salt pork, diced', '6 ounces bacon, cut into small pieces', '1 small onion, minced', '1 ½ cups water, divided', '¼ cup ket

Title: Classic Macaroni Salad
Preparation Time: 270
Serves: 10 serving(s)
Ingredients:['4 cups uncooked elbow macaroni', '1 cup mayonnaise', '1/4 cup distilled white vinegar', '2/3 cup white sugar', '2 1/2 tablespoons prepared yellow mustard', '1 1/2 teaspoons salt', '1/2 teaspoon ground black pepper', '1 large onion, chopped', '2 stalks celery, chopped', '1 green bell pepper, seeded and chopped', '1/4 cup grated carrot (optional)', '2 tablespoons chopped pimento peppers (optional)']
Instructions: Bring a large pot of lightly salted water to a boil. Add the macaroni, and cook until tender, about 8 minutes. Rinse under cold water and drain.
In a large bowl, mix together the mayonnaise, vinegar, sugar, mustard, salt and pepper. Stir in the onion, celery, green pepper, carrot, pimentos and macaroni. Refrigerate for at least 4 hours before serving, but preferably overnight.
Image url: https://images.media-allrecipes.com/userphotos/560x315/878056.jpg
Category: 4th of July
Title: Honey Musta

Title: Simple BBQ Ribs
Preparation Time: 120
Serves: None
Ingredients:['2 ½ pounds country style pork ribs', '1 tablespoon garlic powder', '1 teaspoon ground black pepper', '2 tablespoons salt', '1 cup barbeque sauce']
Instructions: Place ribs in a large pot with enough water to cover. Season with garlic powder, black pepper and salt. Bring water to a boil, and cook ribs until tender.
Preheat oven to 325 degrees F (165 degrees C).
Remove ribs from pot, and place them in a 9x13 inch baking dish. Pour barbeque sauce over ribs. Cover dish with aluminum foil, and bake in the preheated oven for 1 to 1 1/2 hours, or until internal temperature of pork has reached 160 degrees F (70 degrees C).
Image url: https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.com%2Fuserphotos%2F998440.jpg
Category: 4th of July
Title: Judy's Strawberry Pretzel Salad
Preparation Time: 120
Serves: 16 serving(s)
Ingredients:['1 1/2 cups crushed pretzels', '4 1/2 tablespoons white sug

Title: Firecracker Grilled Alaska Salmon
Preparation Time: 400
Serves: 8 serving(s)
Ingredients:['8 (4 ounce) fillets salmon', '1/2 cup peanut oil', '4 tablespoons soy sauce', '4 tablespoons balsamic vinegar', '4 tablespoons green onions, chopped', '3 teaspoons brown sugar', '2 cloves garlic, minced', '1 1/2 teaspoons ground ginger', '2 teaspoons crushed red pepper flakes', '1 teaspoon sesame oil', '1/2 teaspoon salt']
Instructions: Place salmon filets in a medium, nonporous glass dish. In a separate medium bowl, combine the peanut oil, soy sauce, vinegar, green onions, brown sugar, garlic, ginger, red pepper flakes, sesame oil and salt. Whisk together well, and pour over the fish. Cover and marinate the fish in the refrigerator for 4 to 6 hours.
Prepare an outdoor grill with coals about 5 inches from the grate, and lightly oil the grate.
Grill the fillets 5 inches from coals for 10 minutes per inch of thickness, measured at the thickest part, or until fish just flakes with a fork. Tur

...

Title: Italian Sausage Soup
Preparation Time: 50
Serves: None
Ingredients:['1 pound Italian sausage', '1 clove garlic, minced', '2 (14 ounce) cans beef broth', '1 (14.5 ounce) can Italian-style stewed tomatoes', '1 cup sliced carrots', '1 (14.5 ounce) can great Northern beans, undrained', '2 small zucchini, cubed', '2 cups spinach - packed, rinsed and torn', '¼ teaspoon ground black pepper', '¼ teaspoon salt']
Instructions: In a stockpot or Dutch oven, brown sausage with garlic.  Stir in  broth, tomatoes and carrots, and season with salt and pepper. Reduce heat, cover, and simmer 15 minutes.
Stir in beans with liquid and zucchini. Cover, and simmer another 15 minutes, or until zucchini is tender.
Remove from heat, and add spinach.  Replace lid allowing the heat from the soup to cook the spinach leaves.  Soup is ready to serve after 5 minutes.
Image url: https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.com%2Fuserphotos%2F6285692.jpg
Category: All Wo

Title: Classic Cuban Midnight (Medianoche) Sandwich
Preparation Time: 23
Serves: 4 sandwiches
Ingredients:['4 sweet bread rolls', '½ cup mayonnaise', '¼ cup prepared mustard', '1 pound thinly sliced cooked ham', '1 pound thinly sliced fully cooked pork', '1 pound sliced Swiss cheese', '1 cup dill pickle slices', '2 tablespoons butter, melted']
Instructions: Split the sandwich rolls in half, and spread mustard and mayonnaise liberally onto the cut sides. On each sandwich, place and equal amount of Swiss cheese, ham and pork in exactly that order. Place a few pickles onto each one, and put the top of the roll onto the sandwich. Brush the tops with melted butter.
Press each sandwich in a sandwich press heated to medium-high heat. If a sandwich press is not available, use a large skillet over medium-high heat, and press the sandwiches down using a sturdy plate or skillet. Some indoor grills may be good for this also. Cook for 5 to 8 minutes, keeping sandwiches pressed. If using a skillet, 

Title: Indian-Style Chicken and Onions
Preparation Time: 0
Serves: 4 servings
Ingredients:['3 tablespoons water', '6 cloves garlic', '2 serrano chile peppers, stemmed', '1 (2 inch) piece fresh ginger, peeled and thinly sliced', '¼ cup ghee (clarified butter)', '2 pounds chicken drumsticks', '1 pinch salt and ground black pepper to taste', '1 large white onion, sliced', '1 tablespoon dried fenugreek leaves', '2 teaspoons ground coriander', '1 teaspoon garam masala', '½ teaspoon ground turmeric', '3 tomatoes plum tomatoes, grated', '¾ cup milk', '½ cup heavy whipping cream', '⅓ cup finely chopped fresh cilantro']
Instructions: Puree water, garlic, serrano chile peppers, and ginger together in a blender or food processor until smooth.

Heat ghee in a 6-quart saucepan over medium heat. Season chicken with salt and pepper. Cook chicken in hot ghee until browned, 3 to 4 minutes per side. Transfer chicken to a plate.

Cook and stir garlic puree in the 6-quart saucepan until golden, 2 to 3 min

Title: Annie's Fruit Salsa and Cinnamon Chips
Preparation Time: 45
Serves: None
Ingredients:['2 kiwis, peeled and diced', '2 Golden Delicious apples - peeled, cored and diced', '8 ounces raspberries', '1 pound strawberries', '2 tablespoons white sugar', '1 tablespoon brown sugar', '3 tablespoons fruit preserves, any flavor', '10 (10 inch) flour tortillas', 'butter flavored cooking spray', '2 tablespoons cinnamon sugar']
Instructions: In a large bowl, thoroughly mix kiwis, Golden Delicious apples, raspberries, strawberries, white sugar, brown sugar and fruit preserves. Cover and chill in the refrigerator at least 15 minutes.
Preheat oven to 350 degrees F (175 degrees C).
Coat one side of each flour tortilla with butter flavored cooking spray. Cut into wedges and arrange in a single layer on a large baking sheet. Sprinkle wedges with desired amount of cinnamon sugar. Spray again with cooking spray.
Bake in the preheated oven 8 to 10 minutes. Repeat with any remaining tortilla wedges. All

Title: Awesome Sausage, Apple and Cranberry Stuffing
Preparation Time: 100
Serves: None
Ingredients:['1 ½ cups cubed whole wheat bread', '3 ¾ cups cubed white bread', '1 pound ground turkey sausage', '1 cup chopped onion', '¾ cup chopped celery', '2 ½ teaspoons dried sage', '1 ½ teaspoons dried rosemary', '½ teaspoon dried thyme', '1 Golden Delicious apple, cored and chopped', '¾ cup dried cranberries', '⅓ cup minced fresh parsley', '1 cooked turkey liver, finely chopped', '¾ cup turkey stock', '4 tablespoons unsalted butter, melted']
Instructions: Preheat oven to 350 degree F (175 degree C). Spread the white and whole wheat bread cubes in a single layer on a large baking sheet. Bake for 5 to 7 minutes in the preheated oven, or until evenly toasted. Transfer toasted bread cubes to a large bowl.
In a large skillet, cook the sausage and onions over medium heat, stirring and breaking up the lumps until evenly browned. Add the celery, sage, rosemary, and thyme; cook, stirring, for 2 minute

This is the main code that scrapes individual recipes using scrape_me function from recipe_scrapers library, stores into a variable and bulk updates the MongoDB database. PyMongo library is used to connect to the 'all_recipes' MongoDB database hosted on Azure cloud through MongoDB Atlas. 

In [ ]:
# ensure you update your host information below!
DB_URI = os.environ.get('DATABASE_URI')

# don't update this information
DB_NAME = "all_recipes"
try:
    client = MongoClient(DB_URI, maxPoolSize=50, wtimeout=2500, serverSelectionTimeoutMS=5000)
    print(client.server_info())
    db = client[DB_NAME]
except Exception as e:
    print(str(e))
bulk_updates = []
for cat in tqdm(recipes_unique):
    for url in tqdm(recipes_unique[cat]):
        counter = 0
        try:
            time.sleep(10)
            scraper = scrape_me(url)
            counter += 1
        except Exception as e:
            print(f'Error for {url}')
            print(str(e))
            continue
        if counter == 1000:
            time.sleep(3000)
            counter = 0
            
        recipe_doc = {"title":scraper.title(), 
                      "preparation_time": scraper.total_time(),
                      "serves":scraper.yields(),
                      "ingredients":scraper.ingredients(),
                      "instructions":scraper.instructions(),
                      "image_url":scraper.image(),
                      "category":cat,
                      "ratings":scraper.ratings()
                        }
        bulk_updates.append(UpdateOne(
                    {"title": scraper.title()},
                    {"$set": recipe_doc}, upsert=True))
    print(f'No of documents to insert:{len(bulk_updates)}')
try:
    bulk_results = db.recipes.bulk_write(bulk_updates)
    print(f"{bulk_results.modified_count} documents updated")
except InvalidOperation:
    print("no updates necessary")
except Exception as e:
    print(str(e))       
        